In [14]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import json
import pandas
import pprint

In [40]:
import cv2
vidcap = cv2.VideoCapture('/Users/dancrawford/git/traffic/data/IMG_7616.MOV')
success,image = vidcap.read()
count = 0
success = True
while success:
  success,image = vidcap.read()
#   print('Read a new frame: ', success)
  cv2.imwrite("/Users/dancrawford/git/traffic/data/movie_slice/frame%d.jpg" % count, image)     # save frame as JPEG file
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [45]:
from darkflow.net.build import TFNet
import cv2
import os
import json
import pandas as pd

options = {"model": "./cfg/yolo.cfg", "load": "./bin/yolo.weights", "threshold": 0.1}

tfnet = TFNet(options)

for file in os.listdir('/Users/dancrawford/git/traffic/data/movie_slice/'):
    if(file[0]!="."):
        print(file)
        filename, file_extension = os.path.splitext(file)
        imgcv = cv2.imread("/Users/dancrawford/git/traffic/data/movie_slice/"+file)
        result = tfnet.return_predict(imgcv)
        pd.DataFrame(result).to_json("/Users/dancrawford/git/traffic/data/json/"+filename+'.json', orient='split')

Parsing ./cfg/yolo.cfg
Loading ./bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.561676025390625s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bn

In [51]:
def save_video():
    os.system('ffmpeg -r 30 -i /Users/dancrawford/git/traffic/data/tagged/frame%01d.jpg -vcodec mpeg4 -y /Users/dancrawford/git/traffic/data/tagged/tagged_movie.mp4')

In [52]:
def draw_image_with_json(img_file, result_file):
    out_name = os.path.basename(img_file)
    out_base = os.path.splitext(out_name)
    imgcv = cv2.imread('/Users/dancrawford/git/traffic/data/movie_slice/'+img_file)
    with open(result_file) as json_data:
        result = json.load(json_data)
    
    for r in result['data']:
        if r[2] in ['car', 'bicycle', 'truck', 'motorcycle']:
            if r[1] > 0.35:
                cv2.rectangle(imgcv, (r[0]['x'],r[0]['y']), (r[3]['x'], r[3]['y']), (0, 255, 0), 4) 
                text_x, text_y = r[3]["x"] - 10, r[3]["y"] - 10
                cv2.putText(imgcv, r[2], (text_x, text_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imwrite('/Users/dancrawford/git/traffic/data/tagged/'+out_base[0]+'.jpg',imgcv)

In [53]:
for file_name in os.listdir('/Users/dancrawford/git/traffic/data/movie_slice/'):
    base_name = os.path.basename(file_name)
    f, f_extension = os.path.splitext(base_name)
    draw_image_with_json(file_name, '/Users/dancrawford/git/traffic/data/json/'+f+".json")
save_video()